In [13]:
import numpy as np
import pandas as pd
import os
import sys

project_root = os.path.dirname(os.path.dirname(os.path.abspath('.')))
sys.path.append(project_root)


In [14]:
from practicas import P4

df=pd.read_csv(r'Loan_default.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  object 
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  object 
 11  EmploymentType  255347 non-null  object 
 12  MaritalStatus   255347 non-null  object 
 13  HasMortgage     255347 non-null  object 
 14  HasDependents   255347 non-null  object 
 15  LoanPurpose     255347 non-null  object 
 16  HasCoSigner     255347 non-null  object 
 17  Default   

In [16]:
df.isnull().sum()

LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64

In [17]:
df.columns

Index(['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore',
       'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm',
       'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner',
       'Default'],
      dtype='object')

In [18]:
from practicas.P4 import transformacion

In [19]:
results, resumen = transformacion.iv_woe_categoricas(df, target='Default')

/home/eduardo-castillo/Documentos/diplomado/repositorio/librerias/venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
resumen

,variable,IV
0,LoanID,inf
2,EmploymentType,0.020633
7,HasCoSigner,0.014952
5,HasDependents,0.011747
1,Education,0.008127
3,MaritalStatus,0.007784
4,HasMortgage,0.005096
6,LoanPurpose,0.005057


In [ ]:
### SelectKBest 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import KBinsDiscretizer, MinMaxScaler


### Cambiar por Variable Objetivo
X = df.drop(columns=['Default'])
y = df['Default']

num_vars = X.select_dtypes(include=['int64', 'float64']).columns
X_num = X[num_vars]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_num)
### Discretizamos las varibales numericas
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
X_binned = kbd.fit_transform(X_scaled)

selector = SelectKBest(score_func=chi2, k=7)
X_new = selector.fit_transform(X_binned, y)


selected_features = num_vars[selector.get_support()]

print(f'Las variables seleccionadas son: {selected_features}')


### WOE
woe, iv = transformacion.calcular_woe_iv(df, target='Default')

### eliminacion de las variables 

# Definir umbral de IV mínimo por definicion
umbral_iv = 0.02

# Seleccionar variables con IV >= umbral
variables_seleccionadas = iv[iv['IV'] <= umbral_iv]['variable'].tolist()
### Eliminamos las columnas con bajo nivel predictivo del df

df.drop(columns=variables_seleccionadas,inplace=True)

Las variables seleccionadas son: Index(['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
       'NumCreditLines', 'InterestRate'],
      dtype='object')


/home/eduardo-castillo/Documentos/diplomado/repositorio/librerias/venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [24]:
variables_seleccionadas

['HasCoSigner',
 'HasDependents',
 'Education',
 'MaritalStatus',
 'HasMortgage',
 'LoanPurpose']